In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
from itertools import compress
import random


In [6]:
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)
class_choosen = [0,1,2,3,4]
boolean_array_train = [target in class_choosen for target in training_data.targets]
boolean_array_test = [target in class_choosen for target in test_data.targets]
training_data.data = list(compress(training_data.data,boolean_array_train))
training_data.targets = list(compress(training_data.targets,boolean_array_train))
test_data.data = list(compress(training_data.data,boolean_array_train))
test_data.targets = list(compress(training_data.targets,boolean_array_train))
Task_1 = {}
Task_1['query'] = randomSelectSamples(test_data, 5)
Task_1['support'] = randomSelectSamplesByClasses(training_data, class_choosen,5)

Files already downloaded and verified
Files already downloaded and verified
[4, 0, 1, 2, 2, 4, 2, 2, 1, 1, 0, 4, 3, 0, 3, 4, 2, 0, 1, 1, 4, 0, 3, 3, 3]


In [3]:
print(training_data)

Dataset CIFAR10
    Number of datapoints: 25000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [5]:
# 5 n-way 5 k-shot
def randomSelectSamples(data, samples):

    randomlist = random.sample(range(0, len(data.targets)-1), min(len(data.targets), samples))
    data.targets = [data.targets[i] for i in randomlist]
    data.data = [data.data[i] for i in randomlist]
    
    return data

def randomSelectSamplesByClasses(data, classes, sample_size):

    randomlist = random.sample(range(0, len(data.targets)), min(len(data.targets), 100*len(classes)*sample_size))
    counter_by_class = {}
    new_index_list = []
    for i in randomlist:
        if(data.targets[i] not in counter_by_class):
            counter_by_class[data.targets[i]] = 1
            new_index_list.append(i)
        
        elif(counter_by_class[data.targets[i]] < sample_size):
            counter_by_class[data.targets[i]] += 1
            new_index_list.append(i)
            
    data.targets = [data.targets[i] for i in new_index_list]
    data.data = [data.data[i] for i in new_index_list]
    print(data.targets)
    return data

In [48]:
random.sample(range(0,20), 3)

[15, 10, 1]

In [140]:
import  torch
from    torch import nn
from    torch.nn import functional as F
import  numpy as np



# """Taken From:

# @misc{MAML_Pytorch,
#   author = {Liangqu Long},
#   title = {MAML-Pytorch Implementation},
#   year = {2018},
#   publisher = {GitHub},
#   journal = {GitHub repository},
#   howpublished = {\url{https://github.com/dragen1860/MAML-Pytorch}},
#   commit = {master}
# }

# """
# import torch.nn.functional as F
class Learner(nn.Module):
    def __init__(self, config, imgc, imgsz):
        """
        :param config: network config file, type:list of (string, list)
        :param imgc: 1 or 3
        :param imgsz:  28 or 84
        """
        super(Learner, self).__init__()


        self.config = config

        # this dict contains all tensors needed to be optimized
        self.vars = nn.ParameterList()
        # running_mean and running_var
        self.vars_bn = nn.ParameterList()

        for i, (name, param) in enumerate(self.config):
            if name is 'conv2d':
                # [ch_out, ch_in, kernelsz, kernelsz]
                w = nn.Parameter(torch.ones(*param[:4]))
                # gain=1 according to cbfin's implementation
                torch.nn.init.kaiming_normal_(w)
                self.vars.append(w)
                # [ch_out]
                self.vars.append(nn.Parameter(torch.zeros(param[0])))

            elif name is 'convt2d':
                # [ch_in, ch_out, kernelsz, kernelsz, stride, padding]
                w = nn.Parameter(torch.ones(*param[:4]))
                # gain=1 according to cbfin's implementation
                torch.nn.init.kaiming_normal_(w)
                self.vars.append(w)
                # [ch_in, ch_out]
                self.vars.append(nn.Parameter(torch.zeros(param[1])))

            elif name is 'linear':
                # [ch_out, ch_in]
                w = nn.Parameter(torch.ones(*param))
                # gain=1 according to cbfinn's implementation
                torch.nn.init.kaiming_normal_(w)
                self.vars.append(w)
                # [ch_out]
                self.vars.append(nn.Parameter(torch.zeros(param[0])))

            elif name is 'bn':
                # [ch_out]
                w = nn.Parameter(torch.ones(param[0]))
                self.vars.append(w)
                # [ch_out]
                self.vars.append(nn.Parameter(torch.zeros(param[0])))

                # must set requires_grad=False
                running_mean = nn.Parameter(torch.zeros(param[0]), requires_grad=False)
                running_var = nn.Parameter(torch.ones(param[0]), requires_grad=False)
                self.vars_bn.extend([running_mean, running_var])


            elif name in ['tanh', 'relu', 'upsample', 'avg_pool2d', 'max_pool2d',
                          'flatten', 'reshape', 'leakyrelu', 'sigmoid']:
                continue
            else:
                raise NotImplementedError






    def extra_repr(self):
        info = ''

        for name, param in self.config:
            if name is 'conv2d':
                tmp = 'conv2d:(ch_in:%d, ch_out:%d, k:%dx%d, stride:%d, padding:%d)'\
                      %(param[1], param[0], param[2], param[3], param[4], param[5],)
                info += tmp + '\n'

            elif name is 'convt2d':
                tmp = 'convTranspose2d:(ch_in:%d, ch_out:%d, k:%dx%d, stride:%d, padding:%d)'\
                      %(param[0], param[1], param[2], param[3], param[4], param[5],)
                info += tmp + '\n'

            elif name is 'linear':
                tmp = 'linear:(in:%d, out:%d)'%(param[1], param[0])
                info += tmp + '\n'

            elif name is 'leakyrelu':
                tmp = 'leakyrelu:(slope:%f)'%(param[0])
                info += tmp + '\n'


            elif name is 'avg_pool2d':
                tmp = 'avg_pool2d:(k:%d, stride:%d, padding:%d)'%(param[0], param[1], param[2])
                info += tmp + '\n'
            elif name is 'max_pool2d':
                tmp = 'max_pool2d:(k:%d, stride:%d, padding:%d)'%(param[0], param[1], param[2])
                info += tmp + '\n'
            elif name in ['flatten', 'tanh', 'relu', 'upsample', 'reshape', 'sigmoid', 'use_logits', 'bn']:
                tmp = name + ':' + str(tuple(param))
                info += tmp + '\n'
            else:
                raise NotImplementedError

        return info



    def forward(self, x, vars=None, bn_training=True):
        """
        This function can be called by finetunning, however, in finetunning, we dont wish to update
        running_mean/running_var. Thought weights/bias of bn is updated, it has been separated by fast_weights.
        Indeed, to not update running_mean/running_var, we need set update_bn_statistics=False
        but weight/bias will be updated and not dirty initial theta parameters via fast_weiths.
        :param x: [b, 1, 28, 28]
        :param vars:
        :param bn_training: set False to not update
        :return: x, loss, likelihood, kld
        """

        if vars is None:
            vars = self.vars

        idx = 0
        bn_idx = 0

        for name, param in self.config:
            if name is 'conv2d':
                w, b = vars[idx], vars[idx + 1]
                # remember to keep synchrozied of forward_encoder and forward_decoder!
                x = F.conv2d(x, w, b, stride=param[4], padding=param[5])
                idx += 2
                # print(name, param, '\tout:', x.shape)
            elif name is 'convt2d':
                w, b = vars[idx], vars[idx + 1]
                # remember to keep synchrozied of forward_encoder and forward_decoder!
                x = F.conv_transpose2d(x, w, b, stride=param[4], padding=param[5])
                idx += 2
                # print(name, param, '\tout:', x.shape)
            elif name is 'linear':
                w, b = vars[idx], vars[idx + 1]
                x = F.linear(x, w, b)
                idx += 2
                # print('forward:', idx, x.norm().item())
            elif name is 'bn':
                w, b = vars[idx], vars[idx + 1]
                running_mean, running_var = self.vars_bn[bn_idx], self.vars_bn[bn_idx+1]
                x = F.batch_norm(x, running_mean, running_var, weight=w, bias=b, training=bn_training)
                idx += 2
                bn_idx += 2

            elif name is 'flatten':
                # print(x.shape)
                x = x.view(x.size(0), -1)
            elif name is 'reshape':
                # [b, 8] => [b, 2, 2, 2]
                x = x.view(x.size(0), *param)
            elif name is 'relu':
                x = F.relu(x, inplace=param[0])
            elif name is 'leakyrelu':
                x = F.leaky_relu(x, negative_slope=param[0], inplace=param[1])
            elif name is 'tanh':
                x = F.tanh(x)
            elif name is 'sigmoid':
                x = torch.sigmoid(x)
            elif name is 'upsample':
                x = F.upsample_nearest(x, scale_factor=param[0])
            elif name is 'max_pool2d':
                x = F.max_pool2d(x, param[0], param[1], param[2])
            elif name is 'avg_pool2d':
                x = F.avg_pool2d(x, param[0], param[1], param[2])

            else:
                raise NotImplementedError

        # make sure variable is used properly
        assert idx == len(vars)
        assert bn_idx == len(self.vars_bn)


        return x


    def zero_grad(self, vars=None):
        """
        :param vars:
        :return:
        """
        with torch.no_grad():
            if vars is None:
                for p in self.vars:
                    if p.grad is not None:
                        p.grad.zero_()
            else:
                for p in vars:
                    if p.grad is not None:
                        p.grad.zero_()

    def parameters(self):
        """
        override this function since initial parameters will return with a generator.
        :return:
        """
        return self.vars

<>:40: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:58: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:95: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:100: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:105: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:109: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:114: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:117: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:149: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:155: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:161: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:166: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:173: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:176: SyntaxWarning: "is" with a literal. Did you mean "==

In [166]:
dataloader_spt = DataLoader(Task_1['support'], batch_size = 25)
dataloader_qry = DataLoader(Task_1['query'], batch_size =5)

[[[228 222 236]
  [224 213 222]
  [212 206 207]
  ...
  [141 129 123]
  [ 87  73  71]
  [ 50  43  44]]

 [[233 228 241]
  [224 213 223]
  [215 209 211]
  ...
  [125 113 110]
  [ 58  42  40]
  [ 62  52  48]]

 [[234 229 242]
  [224 213 222]
  [218 212 213]
  ...
  [144 134 132]
  [ 82  65  61]
  [ 99  85  73]]

 ...

 [[108 108 107]
  [119 119 117]
  [125 125 123]
  ...
  [146 149 154]
  [144 147 153]
  [134 137 143]]

 [[119 119 119]
  [122 122 122]
  [124 124 124]
  ...
  [147 149 158]
  [130 133 142]
  [110 113 122]]

 [[130 129 132]
  [130 129 132]
  [125 125 127]
  ...
  [103 107 111]
  [ 60  63  67]
  [ 43  47  51]]]


In [226]:
n_ways = 5
# [ch_in, ch_out, kernelsz, kernelsz, stride, padding]

config = [
    ('conv2d', [25, 3, 3, 3, 2, 0]),
    ('relu', [True]),
    ('bn', [25]),
    ('conv2d', [25, 25, 3, 3, 2, 0]),
    ('relu', [True]),
    ('bn', [25]),
    ('conv2d', [25, 25, 3, 3, 2, 0]),
    ('relu', [True]),
    ('bn', [25]),
    ('conv2d', [25, 25, 2, 2, 1, 0]),
    ('relu', [True]),
    ('bn', [25]),
    ('flatten', []),
    ('linear', [n_ways, 100])
]

In [230]:
model = Learner(config,3,32)
batch = [(dataloader_spt,dataloader_qry)]
meta_lr = 1e-3
meta_opt = torch.optim.SGD(model.parameters(), lr = 1e-3)
for batch in task_batch:    
    outer_loss = 0
    for task in batch:
        train_inputs, train_targets = iter(task[0]).next()
        test_inputs, test_targets = iter(task[1]).next()
        train_logits = model(train_inputs)
        inner_loss = F.cross_entropy(train_logits, train_targets)
        model.zero_grad()
        grad = torch.autograd.grad(inner_loss, model.parameters())
        fast_weights = list(map(lambda p: p[1] - meta_lr * p[0], zip(grad, model.parameters())))

        test_logits = model(test_inputs, fast_weights)
        outer_loss += F.cross_entropy(test_logits, test_targets)

    meta_opt.zero_grad()
    outer_loss.backward()
    meta_opt.step()

torch.Size([25, 3, 32, 32])
